# Quantifying accessibility to urban amenities accross canadian cities

#### Purpose of this notebook
- 1) Download point, polygon, and pedestrian street networks from openstreetmap
- 2) Classify amenities and points of interests into 7 different categories
- 3) Calculate walking distances from each node in the network to the first nearest category
- 4) Calculate average accessibility (walking distance) by census DA (dissemination area)

In [2]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import pandana as pdna
from pandana.loaders import osm
import matplotlib.pyplot as plt
from shapely import geometry
import fiona
%matplotlib inline
import os
os.chdir('C:/Users/Leonardo/Documents/MetroWork/RealEstateData')

In [1]:
# Create a pandana network from a bounding box
MT_network = osm.pdna_network_from_bbox(43.0561, -80.4491, 44.3931, -78.0277, network_type='walk')
MM_network = osm.pdna_network_from_bbox(45.145, -74.3077, 45.9284, -73.1318, network_type='walk')
MV_network = osm.pdna_network_from_bbox(49.0039, -123.4715, 49.5672, -122.4432, network_type='walk')
MO_network = osm.pdna_network_from_bbox(44.9548, -76.4648, 46.0237, -75.1268, network_type='walk')
ME_network = osm.pdna_network_from_bbox(53.0242, -114.9426, 54.0103, -112.6126, network_type='walk')
MC_network = osm.pdna_network_from_bbox(50.2284,-114.8995, 51.6327, -113.0863, network_type='walk')

In [4]:
#load city boundary polygons
MTAREA = gpd.read_file("Boundaries/MTAREA.shp")
MTAREA = ox.project_gdf(MTAREA, to_latlong=True)
MVAREA = gpd.read_file("Boundaries/MVAREA.shp")
MVAREA = ox.project_gdf(MVAREA, to_latlong=True)
MMAREA = gpd.read_file("Boundaries/MMAREA.shp")
MMAREA = ox.project_gdf(MMAREA, to_latlong=True)
MCAREA = gpd.read_file("Boundaries/MCAREA.shp")
MCAREA = ox.project_gdf(MCAREA, to_latlong=True)
MEAREA = gpd.read_file("Boundaries/MEAREA.shp")
MEAREA = ox.project_gdf(MEAREA, to_latlong=True)
MOAREA = gpd.read_file("Boundaries/MOAREA.shp")
MOAREA = ox.project_gdf(MOAREA, to_latlong=True)

In [5]:
def create_poi_gdf(MAREA = None):

    ##public transport
    #get public transport (rail) network of metro vancouver
    rail = ox.graph_from_polygon(MAREA.iloc[0,1], network_type='all',
                                  infrastructure='way["railway"]')
    #make rail pois
    rail_gdf = ox.save_load.graph_to_gdfs(rail, nodes=True, edges=False, node_geometry=True, fill_edge_geometry=True)
    rail_gdf['amenity'] = 'transit_stop'
    rail_gdf = rail_gdf[['amenity', 'geometry']]

    #get public bus stops
    bus = ox.footprints.footprints_from_polygon(polygon = MAREA.iloc[0,1], footprint_type = 'public_transport')
    bus['amenity'] = 'public_transport'
    bus['centroid'] = bus.centroid
    bus = bus.set_geometry('centroid')
    bus = bus[['amenity', 'centroid']]
    bus = bus.rename(columns = {"centroid": "geometry"})

    ##amenities pois
    # define your selected amenities
    amenities = ["transit_stop", "bus_station", "bus_stop", "bicycle_parking", "gym", "park", "pub", "bar", "theatre", 
                 "cinema", "nightclub", "events_venue", "restaurant", "cafe", "food_court", "marketplace", "community_centre", 
                 "library", "social_facility", "social_centre", "townhall", "school", "childcare", "child_care", "kindergarten",
                 "university", "college", "pharmacy", "dentist", "clinic", "hospital", "doctors", "bank"]

    #request amenities from the OpenStreetMap API (Overpass)
    pois = ox.pois_from_polygon(MAREA.iloc[0,1])
    pois = pois[pois['amenity'].isin(amenities)]
    pois = pois[['amenity','geometry']]

    #get parks and "leisure" elements
    leisure = ["fitness_centre", "sports_centre", "park", "pitch", "playground", "swimming_pool", "garden", "golf_course", "sports_centre", 
               "ice_rink", "dog_park", "nature_reserve", "fitness_centre", "marina", "recreation_ground", "fitness_station", "skate_park"]

    parks = ox.footprints.footprints_from_polygon(polygon = MAREA.iloc[0,1], footprint_type = 'leisure')
    parks = parks[parks['leisure'].isin(leisure)]
    parks['centroid'] = parks.centroid
    parks = parks.set_geometry('centroid')
    parks = parks[['leisure','centroid']]
    parks = parks.rename(columns = {"leisure":"amenity","centroid": "geometry"})

    #merge all the dataframes together
    #merge dataframes together
    pois_list = [pois,rail_gdf, bus, parks]
    pois_all = pd.concat(pois_list, axis=0, ignore_index=True)

    #aggregate all amenities by category
    mobility = ["transit_stop", "bus_station", "bus_stop", "public_transport"]

    active_living = ["bicycle_parking", "gym", "fitness_centre", "sports_centre", "park", "pitch", "playground", "swimming_pool", 
                     "garden", "golf_course", "sports_centre","ice_rink", "dog_park", "nature_reserve", "fitness_centre", "marina", 
                     "recreation_ground", "fitness_station", "skate_park"]

    nightlife = ["pub", "bar", "theatre", "cinema", "nightclub", "events_venue"]

    food_choices = ["restaurant", "cafe", "food_court", "marketplace"]

    community_space = ["community_centre", "library", "social_facility", "social_centre", "townhall"]

    education = ["school", "childcare", "child_care", "kindergarten", "university", "college"]

    health_wellbeing = ["pharmacy", "dentist", "clinic", "hospital", "doctors", "bank"]

    cat_list = [mobility, active_living, nightlife, food_choices, community_space, education, health_wellbeing]
    cat_list_str = ["mobility", "active_living", "nightlife", "food_choices", "community_space", "education", "health_wellbeing"]

    for cat in cat_list:
        cat_index = cat_list.index(cat)
        pois_all.amenity[pois_all['amenity'].isin(cat)] = cat_list_str[cat_index]

    #create x and y columns for network analysis
    pois_all['x'] = pois_all.centroid.x
    pois_all['y'] = pois_all.centroid.y
    
    pois_all['centroid'] = pois_all.centroid
    pois_all = pois_all.set_geometry('centroid')
    pois_all = pois_all[['amenity', 'x', 'y','centroid']]
    pois_all = pois_all.rename(columns = {"centroid": "geometry"})
    pois_all = pois_all.set_geometry('geometry')
    
    return pois_all

In [6]:
#download pois for all cities
MTAREA_pois = create_poi_gdf(MAREA = MTAREA)
MMAREA_pois = create_poi_gdf(MAREA = MMAREA)
MVAREA_pois = create_poi_gdf(MAREA = MVAREA)
MCAREA_pois = create_poi_gdf(MAREA = MCAREA)
MEAREA_pois = create_poi_gdf(MAREA = MEAREA)
MOAREA_pois = create_poi_gdf(MAREA = MOAREA)

C:\Users\Leonardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Leonardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Leonardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Leonardo\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning

In [26]:
MTAREA_pois_df = MTAREA_pois[['amenity', 'x', 'y']]
MMAREA_pois_df = MMAREA_pois[['amenity', 'x', 'y']]
MVAREA_pois_df = MVAREA_pois[['amenity', 'x', 'y']]
MCAREA_pois_df = MCAREA_pois[['amenity', 'x', 'y']]
MEAREA_pois_df = MEAREA_pois[['amenity', 'x', 'y']]
MOAREA_pois_df = MOAREA_pois[['amenity', 'x', 'y']]

In [30]:
#save pois for each city so we dont have to download them again
MTAREA_pois_df.to_csv("access_cleanfiles/pois/tables/MTAREA_pois.csv")
MMAREA_pois_df.to_csv("access_cleanfiles/pois/tables/MMAREA_pois.csv")
MVAREA_pois_df.to_csv("access_cleanfiles/pois/tables/MVAREA_pois.csv")
MCAREA_pois_df.to_csv("access_cleanfiles/pois/tables/MCAREA_pois.csv")
MEAREA_pois_df.to_csv("access_cleanfiles/pois/tables/MEAREA_pois.csv")
MOAREA_pois_df.to_csv("access_cleanfiles/pois/tables/MOAREA_pois.csv")

In [70]:
#save pois for each city so we dont have to download them again
MTAREA_pois.to_file("access_cleanfiles/pois/points/MTAREA_pois.shp")
MMAREA_pois.to_file("access_cleanfiles/pois/points/MMAREA_pois.shp")
MVAREA_pois.to_file("access_cleanfiles/pois/points/MVAREA_pois.shp")
MCAREA_pois.to_file("access_cleanfiles/pois/points/MCAREA_pois.shp")
MEAREA_pois.to_file("access_cleanfiles/pois/points/MEAREA_pois.shp")
MOAREA_pois.to_file("access_cleanfiles/pois/points/MOAREA_pois.shp")

In [176]:
#function to compute accessibility indicator for each category with regards to each node on the network
def create_access_gdf(MAREA_pois = None, network = None, MAREA = None):

    cat_list_str = ["mobility", "active_living", "nightlife", "food_choices", "community_space", "education", "health_wellbeing"]

    #create dummy dataframe (only way of doing it so far is to run dummy network analysis at 1m)
    for cat in cat_list_str:
        pois_subset = MAREA_pois[MAREA_pois['amenity']==cat]
        network.set_pois(category = cat, maxdist = 1, maxitems=len(pois_subset), x_col=pois_subset['x'], y_col=pois_subset['y'])
        accessibility = network.nearest_pois(distance=1, category=cat) 
        
    #now calculate distances
    for cat in cat_list_str:
        pois_subset = MAREA_pois[MAREA_pois['amenity']==cat]
        network.set_pois(category = cat, maxdist = 10000, maxitems=len(pois_subset), 
                                 x_col=pois_subset['x'], y_col=pois_subset['y'])

        accessibility[str(cat)] = network.nearest_pois(distance=10000, category=cat) 

    accessibility = accessibility[["mobility", "active_living", "nightlife", "food_choices", 
                                        "community_space", "education", "health_wellbeing"]]

    #get walking network of Toronto
    walk = ox.graph_from_polygon(MAREA.iloc[0,1], network_type='walk')
    #convert walking network to tuple with nodes and edges gdfs
    walk_gdf = ox.save_load.graph_to_gdfs(walk, nodes=True, edges=False, node_geometry=True, fill_edge_geometry=True)

    import os
    os.chdir(r"C:\Users\Leonardo\Documents\MetroWork")
    #merge accessibility values with walk nodes ids geodataframe
    access = pd.merge(accessibility,
                               walk_gdf,
                               left_on='id',
                               right_on='osmid',
                               how='right')
    #convert to geodataframe
    access = gpd.GeoDataFrame(access, geometry=gpd.points_from_xy(access.x, access.y))
    #set right crs
    access.crs = {'init' :'epsg:4326'}

    access = access[['osmid', "mobility", "active_living", "nightlife", "food_choices",
                             "community_space", "education", "health_wellbeing", 'y', 'x', 'geometry']]

    return access

In [196]:
#compute accessibility indicator for each category for each metro area
MT_access = create_access_gdf(MAREA_pois = MTAREA_pois, network = MT_network, MAREA = MTAREA)
MM_access = create_access_gdf(MAREA_pois = MMAREA_pois, network = MM_network, MAREA = MMAREA)
MV_access = create_access_gdf(MAREA_pois = MVAREA_pois, network = MV_network, MAREA = MVAREA)
MC_access = create_access_gdf(MAREA_pois = MCAREA_pois, network = MC_network, MAREA = MCAREA)
ME_access = create_access_gdf(MAREA_pois = MEAREA_pois, network = ME_network, MAREA = MEAREA)
MO_access = create_access_gdf(MAREA_pois = MOAREA_pois, network = MO_network, MAREA = MOAREA)

In [198]:
#save geodataframes to file
os.chdir('C:/Users/Leonardo/Documents/MetroWork/RealEstateData')
MT_access.to_file("access_cleanfiles/pois/points_access/MT_access.shp")
MM_access.to_file("access_cleanfiles/pois/points_access/MM_access.shp")
MV_access.to_file("access_cleanfiles/pois/points_access/MV_access.shp")
MC_access.to_file("access_cleanfiles/pois/points_access/MC_access.shp")
ME_access.to_file("access_cleanfiles/pois/points_access/ME_access.shp")
MO_access.to_file("access_cleanfiles/pois/points_access/MO_access.shp")

### Create Census Layer with accessibility metrics

In [202]:
##interpolate points in census blocks
import os
os.chdir(r"C:\Users\Leonardo\Documents\MetroWork") 
MTcensus = gpd.read_file("RealEstateData/CensusData2016_MA/CensusBlocks/MTBLOCKS.shp")
MMcensus = gpd.read_file("RealEstateData/CensusData2016_MA/CensusBlocks/MMBLOCKS.shp")
MVcensus = gpd.read_file("RealEstateData/CensusData2016_MA/CensusBlocks/MVBLOCKS.shp")
MEcensus = gpd.read_file("RealEstateData/CensusData2016_MA/CensusBlocks/MEBLOCKS.shp")
MCcensus = gpd.read_file("RealEstateData/CensusData2016_MA/CensusBlocks/MCBLOCKS.shp")
MOcensus = gpd.read_file("RealEstateData/CensusData2016_MA/CensusBlocks/MOBLOCKS.shp")

In [200]:
#function to create accessibility by census block
def create_census_access(access = None, census = None):
    #join transit access points to census
    census_access = gpd.sjoin(census, access, how='left')

    #group-by OBJECTID and calculate mean price_m2 per census block
    census_access['index'] = census_access['DAUID'] 
    census_access = census_access.groupby('DAUID').mean()
    census_access = census_access[["mobility", "active_living", "nightlife", "food_choices",
                                   "community_space", "education", "health_wellbeing"]]
    census_access = census_access.fillna(method = 'ffill')
    census_access = census.merge(census_access, on='DAUID')
    census_access = census_access[["DAUID","mobility", "active_living", "nightlife", "food_choices",
                                   "community_space", "education", "health_wellbeing", "geometry"]]

    return census_access

In [207]:
# access to amenities by census dissemination area
MT_census_access = create_census_access(access = MT_access, census = MTcensus)
MM_census_access = create_census_access(access = MM_access, census = MMcensus)
MV_census_access = create_census_access(access = MV_access, census = MVcensus)
MC_census_access = create_census_access(access = MC_access, census = MCcensus)
ME_census_access = create_census_access(access = ME_access, census = MEcensus)
MO_census_access = create_census_access(access = MO_access, census = MOcensus)

In [208]:
MT_census_access.to_file("RealEstateData/access_cleanfiles/MT_DA_walk_to_amenities.shp")
MM_census_access.to_file("RealEstateData/access_cleanfiles/poly/MM_DA_walk_to_amenities.shp")
MV_census_access.to_file("RealEstateData/access_cleanfiles/poly/MV_DA_walk_to_amenities.shp")
MC_census_access.to_file("RealEstateData/access_cleanfiles/poly/MC_DA_walk_to_amenities.shp")
ME_census_access.to_file("RealEstateData/access_cleanfiles/poly/ME_DA_walk_to_amenities.shp")
MO_census_access.to_file("RealEstateData/access_cleanfiles/poly/MO_DA_walk_to_amenities.shp")